In [ ]:
from mido import Message, MidiFile, MidiTrack, MetaMessage
import stilus.midi.utils as utl
import stilus.models as m
import stilus.midi.imagine as img
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader

In [ ]:
#import a midi file
file_path = "midi/training/mozart/mz_545_3.mid"
tokens = file_path.split("/")
file_name = tokens[len(tokens)-1]
file_name_no_ext = file_name.split(".")[0]

mid = MidiFile(file_path)
file_name_no_ext

In [ ]:
# print all events
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        #if msg.type == "note_on":
        print(msg)

In [ ]:
timeseries_tensor = utl.convert_midi_to_time_series(mid,5,5,8)
print(timeseries_tensor.shape)
n = 136
print(timeseries_tensor[:,n-32:n])

In [ ]:
version ="version_5"
epochs = "5"
net = m.ConvNet_1_0_2().load_from_checkpoint("tb_logs/ConvNet_1_0_2_mozart/"+ version +"/checkpoints/epoch="+ epochs +".ckpt")
#net = m.TransformerNet_1_0_2().load_from_checkpoint("tb_logs/TransformerNet_1_0_2_mozart_test/"+ version +"/checkpoints/epoch="+ epochs +".ckpt")
net.eval( )

In [ ]:
#set path to dataloaders
net.set_data_path("data/mozart_test")

In [ ]:
timeseries_tensor.shape

In [ ]:
generated_series = img.imagine_midi(timeseries_tensor, net, ["128:160", "200:280", "400:448", "1200:1280", "1600:1760","2000:2160"])
print(generated_series[:,136:172])
print(generated_series.shape)

In [ ]:
out_path = 'midi/weight_analysis/' + file_name_no_ext +"_" + net.name + "_" + version + '.mid'
utl.write_midi_from_series(generated_series, 2, out_path )